In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
class SimpleNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleNN, self).__init__()
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.fc1 = nn.Linear(self.input_size , self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x):
        x = x.view(x.size(0), -1) 
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [5]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    return avg_loss, accuracy

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return avg_loss, accuracy


In [6]:
def plot_accuracies(train_accs, test_accs):
    plt.figure(figsize=(10, 6))
    plt.plot(train_accs, label='Train Accuracy', color='blue')
    plt.plot(test_accs, label='Test Accuracy', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Testing Accuracies over Epochs')
    plt.legend()
    plt.show()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 28
hidden_size = 128
output_size = 10
model = SimpleNN(input_size,hidden_size,output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x784 and 28x128)

In [ ]:
train_accuracies = []
test_accuracies = []

num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
    print(f"\nEpoch {epoch} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    train_accuracies.append(train_acc)

    test_loss, test_acc = test(model, device, test_loader, criterion)
    print(f"Epoch {epoch} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")
    test_accuracies.append(test_acc)

plot_accuracies(train_accuracies, test_accuracies)
